# Churn Modelling

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Load the dataset

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.shape

(10000, 14)

In [3]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
#split
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [5]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [6]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [8]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [10]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [13]:
from tensorflow.keras.models import Sequential

# Initializing the ANN
ann = Sequential()

### Adding the first hidden layer

In [14]:
# 6 units and relu activation function
from tensorflow.keras.layers import Dense

# Adding the first hidden layer
ann.add(Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [15]:
# 6 units and relu activation function
# Adding the second hidden layer
ann.add(Dense(units=6, activation='relu'))

### Adding the output layer

In [16]:
# 1 units and sigmoid activation function
# Adding the output layer
ann.add(Dense(units=1, activation='sigmoid'))


## Part 3 - Training the ANN

### Compiling the ANN

In [17]:
#adam optimiser and binary_crossentropy loss function
# Compiling the ANN
ann.compile(optimizer='adam', loss='binary_crossentropy')

### Training the ANN on the Training set

In [18]:
# batch_size = 32, epochs = 100
# Training the ANN on the training set
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 670us/step - loss: 0.5265
Epoch 2/100
250/250 [==============================] - 0s 639us/step - loss: 0.4689
Epoch 3/100
250/250 [==============================] - 0s 638us/step - loss: 0.4430
Epoch 4/100
250/250 [==============================] - 0s 641us/step - loss: 0.4242
Epoch 5/100
250/250 [==============================] - 0s 634us/step - loss: 0.4081
Epoch 6/100
250/250 [==============================] - 0s 641us/step - loss: 0.3957
Epoch 7/100
250/250 [==============================] - 0s 641us/step - loss: 0.3856
Epoch 8/100
250/250 [==============================] - 0s 642us/step - loss: 0.3781
Epoch 9/100
250/250 [==============================] - 0s 641us/step - loss: 0.3724
Epoch 10/100
250/250 [==============================] - 0s 639us/step - loss: 0.3683
Epoch 11/100
250/250 [==============================] - 0s 644us/step - loss: 0.3650
Epoch 12/100
250/250 [==============================] - 0s 661us/step - lo

## Part 4 - Tuning batch and epochs

using GridSerachCV and RandomizedSearchCV try to find the best values related to the epochs and batch_size hyperparameters**

In [19]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [20]:
# Create a function that returns the Keras model
def create_model():
    ann = Sequential()
    ann.add(Dense(units=6, activation='relu'))
    ann.add(Dense(units=6, activation='relu'))
    ann.add(Dense(units=1, activation='sigmoid'))
    ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return ann

In [21]:
# Create a wrapped version of the Keras model
wrapped_model = KerasClassifier(build_fn=create_model)

C:\Users\MSI\AppData\Local\Temp\ipykernel_17740\72691884.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  wrapped_model = KerasClassifier(build_fn=create_model)


In [22]:
param_grid = {
    'epochs': [50, 100, 150],
    'batch_size': [16, 32, 64]
}

In [23]:
# Create the GridSearchCV object
search = GridSearchCV(estimator=wrapped_model, param_grid=param_grid, scoring='accuracy', cv=5)

In [24]:
search.fit(X_train, y_train)

Epoch 1/50
400/400 [==============================] - 1s 674us/step - loss: 0.5950 - accuracy: 0.7234
Epoch 2/50
400/400 [==============================] - 0s 665us/step - loss: 0.4659 - accuracy: 0.7981
Epoch 3/50
400/400 [==============================] - 0s 684us/step - loss: 0.4257 - accuracy: 0.8083
Epoch 4/50
400/400 [==============================] - 0s 680us/step - loss: 0.4027 - accuracy: 0.8234
Epoch 5/50
400/400 [==============================] - 0s 672us/step - loss: 0.3870 - accuracy: 0.8355
Epoch 6/50
400/400 [==============================] - 0s 686us/step - loss: 0.3749 - accuracy: 0.8413
Epoch 7/50
400/400 [==============================] - 0s 692us/step - loss: 0.3663 - accuracy: 0.8464
Epoch 8/50
400/400 [==============================] - 0s 677us/step - loss: 0.3608 - accuracy: 0.8489
Epoch 9/50
400/400 [==============================] - 0s 661us/step - loss: 0.3569 - accuracy: 0.8512
Epoch 10/50
400/400 [==============================] - 0s 666us/step - loss: 0.354

400/400 [==============================] - 1s 1ms/step - loss: 0.3615 - accuracy: 0.8403
Epoch 11/50
400/400 [==============================] - 0s 1ms/step - loss: 0.3583 - accuracy: 0.8458
Epoch 12/50
400/400 [==============================] - 1s 1ms/step - loss: 0.3556 - accuracy: 0.8481
Epoch 13/50
400/400 [==============================] - 0s 1ms/step - loss: 0.3535 - accuracy: 0.8497
Epoch 14/50
400/400 [==============================] - 0s 1ms/step - loss: 0.3518 - accuracy: 0.8505
Epoch 15/50
400/400 [==============================] - 0s 1ms/step - loss: 0.3501 - accuracy: 0.8512
Epoch 16/50
400/400 [==============================] - 0s 1ms/step - loss: 0.3490 - accuracy: 0.8533
Epoch 17/50
400/400 [==============================] - 1s 1ms/step - loss: 0.3478 - accuracy: 0.8539
Epoch 18/50
400/400 [==============================] - 1s 1ms/step - loss: 0.3466 - accuracy: 0.8547
Epoch 19/50
400/400 [==============================] - 0s 1ms/step - loss: 0.3456 - accuracy: 0.8573
Ep

400/400 [==============================] - 1s 1ms/step - loss: 0.3325 - accuracy: 0.8655
Epoch 71/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3321 - accuracy: 0.8655
Epoch 72/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3320 - accuracy: 0.8662
Epoch 73/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3317 - accuracy: 0.8653
Epoch 74/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3321 - accuracy: 0.8655
Epoch 75/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3315 - accuracy: 0.8631
Epoch 76/100
400/400 [==============================] - 1s 2ms/step - loss: 0.3313 - accuracy: 0.8653
Epoch 77/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3312 - accuracy: 0.8645
Epoch 78/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3312 - accuracy: 0.8652
Epoch 79/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3311 - accuracy: 

400/400 [==============================] - 1s 2ms/step - loss: 0.3337 - accuracy: 0.8619
Epoch 51/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3340 - accuracy: 0.8619
Epoch 52/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3336 - accuracy: 0.8611
Epoch 53/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3336 - accuracy: 0.8625
Epoch 54/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3333 - accuracy: 0.8598
Epoch 55/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3330 - accuracy: 0.8633
Epoch 56/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3325 - accuracy: 0.8619
Epoch 57/100
400/400 [==============================] - 0s 1ms/step - loss: 0.3326 - accuracy: 0.8614
Epoch 58/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3325 - accuracy: 0.8592
Epoch 59/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3329 - accuracy: 

Epoch 30/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3429 - accuracy: 0.8594
Epoch 31/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3428 - accuracy: 0.8611
Epoch 32/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3426 - accuracy: 0.8598
Epoch 33/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3420 - accuracy: 0.8606
Epoch 34/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3428 - accuracy: 0.8620
Epoch 35/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3421 - accuracy: 0.8608
Epoch 36/100
400/400 [==============================] - 1s 2ms/step - loss: 0.3418 - accuracy: 0.8631
Epoch 37/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3416 - accuracy: 0.8619
Epoch 38/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3419 - accuracy: 0.8627
Epoch 39/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3412

...

100/100 [==============================] - 0s 2ms/step - loss: 0.3383 - accuracy: 0.8636
Epoch 94/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3382 - accuracy: 0.8619
Epoch 95/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3385 - accuracy: 0.8631
Epoch 96/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3382 - accuracy: 0.8620
Epoch 97/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3377 - accuracy: 0.8630
Epoch 98/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3383 - accuracy: 0.8619
Epoch 99/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3379 - accuracy: 0.8622
Epoch 100/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3375 - accuracy: 0.8628
Epoch 101/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3379 - accuracy: 0.8625
Epoch 102/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3377 - accurac

100/100 [==============================] - 0s 2ms/step - loss: 0.3395 - accuracy: 0.8611
Epoch 103/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3394 - accuracy: 0.8619
Epoch 104/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3391 - accuracy: 0.8616
Epoch 105/150
100/100 [==============================] - 0s 1ms/step - loss: 0.3392 - accuracy: 0.8616
Epoch 106/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3386 - accuracy: 0.8612
Epoch 107/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3381 - accuracy: 0.8636
Epoch 108/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3375 - accuracy: 0.8623
Epoch 109/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3373 - accuracy: 0.8633
Epoch 110/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3371 - accuracy: 0.8634
Epoch 111/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3366 - a

100/100 [==============================] - 0s 2ms/step - loss: 0.3333 - accuracy: 0.8611
Epoch 112/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3332 - accuracy: 0.8614
Epoch 113/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3329 - accuracy: 0.8627
Epoch 114/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3331 - accuracy: 0.8612
Epoch 115/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3328 - accuracy: 0.8637
Epoch 116/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3329 - accuracy: 0.8612
Epoch 117/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3326 - accuracy: 0.8617
Epoch 118/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3327 - accuracy: 0.8623
Epoch 119/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3325 - accuracy: 0.8623
Epoch 120/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3323 - a

500/500 [==============================] - 1s 2ms/step - loss: 0.3383 - accuracy: 0.8625
Epoch 121/150
500/500 [==============================] - 1s 2ms/step - loss: 0.3388 - accuracy: 0.8614
Epoch 122/150
500/500 [==============================] - 1s 2ms/step - loss: 0.3382 - accuracy: 0.8633
Epoch 123/150
500/500 [==============================] - 1s 2ms/step - loss: 0.3379 - accuracy: 0.8625
Epoch 124/150
500/500 [==============================] - 1s 2ms/step - loss: 0.3382 - accuracy: 0.8636
Epoch 125/150
500/500 [==============================] - 1s 2ms/step - loss: 0.3380 - accuracy: 0.8602
Epoch 126/150
500/500 [==============================] - 1s 2ms/step - loss: 0.3378 - accuracy: 0.8630
Epoch 127/150
500/500 [==============================] - 1s 2ms/step - loss: 0.3379 - accuracy: 0.8600
Epoch 128/150
500/500 [==============================] - 1s 2ms/step - loss: 0.3378 - accuracy: 0.8604
Epoch 129/150
500/500 [==============================] - 1s 2ms/step - loss: 0.3377 - a

GridSearchCV(cv=5,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000002049293A880>,
             param_grid={'batch_size': [16, 32, 64], 'epochs': [50, 100, 150]},
             scoring='accuracy')

In [25]:
# Access the best parameters
best_params = search.best_params_
print("Best Parameters:", best_params)

Best Parameters: {'batch_size': 16, 'epochs': 150}


In [26]:
# Access the best score
best_score = search.best_score_
print("Best score:", best_score)

Best score: 0.86075


In [27]:
# Create the RandomizedSearchCV object
search1 = RandomizedSearchCV(estimator=wrapped_model, param_distributions=param_grid, scoring='accuracy', cv=5, n_iter=10)

In [28]:
search1.fit(X_train, y_train)

Epoch 1/50


C:\Users\MSI\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


400/400 [==============================] - 1s 2ms/step - loss: 0.5994 - accuracy: 0.6942
Epoch 2/50
400/400 [==============================] - 1s 2ms/step - loss: 0.4524 - accuracy: 0.8112
Epoch 3/50
400/400 [==============================] - 1s 2ms/step - loss: 0.4203 - accuracy: 0.8195
Epoch 4/50
400/400 [==============================] - 1s 2ms/step - loss: 0.4023 - accuracy: 0.8258
Epoch 5/50
400/400 [==============================] - 1s 2ms/step - loss: 0.3881 - accuracy: 0.8355
Epoch 6/50
400/400 [==============================] - 1s 2ms/step - loss: 0.3781 - accuracy: 0.8384
Epoch 7/50
400/400 [==============================] - 1s 2ms/step - loss: 0.3694 - accuracy: 0.8442
Epoch 8/50
400/400 [==============================] - 1s 2ms/step - loss: 0.3634 - accuracy: 0.8469
Epoch 9/50
400/400 [==============================] - 1s 2ms/step - loss: 0.3589 - accuracy: 0.8480
Epoch 10/50
400/400 [==============================] - 1s 2ms/step - loss: 0.3560 - accuracy: 0.8498
Epoch 11/5

400/400 [==============================] - 1s 2ms/step - loss: 0.4045 - accuracy: 0.8273
Epoch 12/50
400/400 [==============================] - 1s 2ms/step - loss: 0.4017 - accuracy: 0.8286
Epoch 13/50
400/400 [==============================] - 1s 2ms/step - loss: 0.3990 - accuracy: 0.8308
Epoch 14/50
400/400 [==============================] - 1s 2ms/step - loss: 0.3949 - accuracy: 0.8305
Epoch 15/50
400/400 [==============================] - 1s 2ms/step - loss: 0.3910 - accuracy: 0.8350
Epoch 16/50
400/400 [==============================] - 1s 2ms/step - loss: 0.3854 - accuracy: 0.8380
Epoch 17/50
400/400 [==============================] - 1s 2ms/step - loss: 0.3795 - accuracy: 0.8416
Epoch 18/50
400/400 [==============================] - 1s 2ms/step - loss: 0.3722 - accuracy: 0.8436
Epoch 19/50
400/400 [==============================] - 1s 2ms/step - loss: 0.3665 - accuracy: 0.8464
Epoch 20/50
400/400 [==============================] - 1s 2ms/step - loss: 0.3613 - accuracy: 0.8489
Ep

400/400 [==============================] - 1s 2ms/step - loss: 0.3402 - accuracy: 0.8591
Epoch 72/100
400/400 [==============================] - 1s 2ms/step - loss: 0.3395 - accuracy: 0.8608
Epoch 73/100
400/400 [==============================] - 1s 2ms/step - loss: 0.3394 - accuracy: 0.8600
Epoch 74/100
400/400 [==============================] - 1s 2ms/step - loss: 0.3392 - accuracy: 0.8592
Epoch 75/100
400/400 [==============================] - 1s 2ms/step - loss: 0.3395 - accuracy: 0.8592
Epoch 76/100
400/400 [==============================] - 1s 2ms/step - loss: 0.3393 - accuracy: 0.8594
Epoch 77/100
400/400 [==============================] - 1s 2ms/step - loss: 0.3391 - accuracy: 0.8592
Epoch 78/100
400/400 [==============================] - 1s 2ms/step - loss: 0.3393 - accuracy: 0.8584
Epoch 79/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3387 - accuracy: 0.8606
Epoch 80/100
400/400 [==============================] - 1s 1ms/step - loss: 0.3390 - accuracy: 

...

100/100 [==============================] - 0s 2ms/step - loss: 0.3435 - accuracy: 0.8600
Epoch 99/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3431 - accuracy: 0.8605
Epoch 100/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3430 - accuracy: 0.8627
Epoch 101/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3427 - accuracy: 0.8625
Epoch 102/150
100/100 [==============================] - 0s 1ms/step - loss: 0.3431 - accuracy: 0.8619
Epoch 103/150
100/100 [==============================] - 0s 1ms/step - loss: 0.3430 - accuracy: 0.8616
Epoch 104/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3428 - accuracy: 0.8623
Epoch 105/150
100/100 [==============================] - 0s 1ms/step - loss: 0.3425 - accuracy: 0.8627
Epoch 106/150
100/100 [==============================] - 0s 1ms/step - loss: 0.3426 - accuracy: 0.8598
Epoch 107/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3423 - ac

100/100 [==============================] - 0s 1ms/step - loss: 0.3294 - accuracy: 0.8666
Epoch 108/150
100/100 [==============================] - 0s 1ms/step - loss: 0.3294 - accuracy: 0.8652
Epoch 109/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3292 - accuracy: 0.8656
Epoch 110/150
100/100 [==============================] - 0s 1ms/step - loss: 0.3292 - accuracy: 0.8662
Epoch 111/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3294 - accuracy: 0.8670
Epoch 112/150
100/100 [==============================] - 0s 1ms/step - loss: 0.3290 - accuracy: 0.8673
Epoch 113/150
100/100 [==============================] - 0s 1ms/step - loss: 0.3290 - accuracy: 0.8667
Epoch 114/150
100/100 [==============================] - 0s 1ms/step - loss: 0.3290 - accuracy: 0.8653
Epoch 115/150
100/100 [==============================] - 0s 1ms/step - loss: 0.3292 - accuracy: 0.8669
Epoch 116/150
100/100 [==============================] - 0s 1ms/step - loss: 0.3292 - a

100/100 [==============================] - 0s 2ms/step - loss: 0.3334 - accuracy: 0.8661
Epoch 117/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3336 - accuracy: 0.8655
Epoch 118/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3334 - accuracy: 0.8645
Epoch 119/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3333 - accuracy: 0.8644
Epoch 120/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3331 - accuracy: 0.8628
Epoch 121/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3334 - accuracy: 0.8658
Epoch 122/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3331 - accuracy: 0.8647
Epoch 123/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3330 - accuracy: 0.8656
Epoch 124/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3328 - accuracy: 0.8652
Epoch 125/150
100/100 [==============================] - 0s 2ms/step - loss: 0.3331 - a

500/500 [==============================] - 1s 2ms/step - loss: 0.3313 - accuracy: 0.8640
Epoch 126/150
500/500 [==============================] - 1s 2ms/step - loss: 0.3310 - accuracy: 0.8626
Epoch 127/150
500/500 [==============================] - 1s 1ms/step - loss: 0.3311 - accuracy: 0.8649
Epoch 128/150
500/500 [==============================] - 1s 1ms/step - loss: 0.3310 - accuracy: 0.8625
Epoch 129/150
500/500 [==============================] - 1s 2ms/step - loss: 0.3309 - accuracy: 0.8626
Epoch 130/150
500/500 [==============================] - 1s 2ms/step - loss: 0.3309 - accuracy: 0.8641
Epoch 131/150
500/500 [==============================] - 1s 2ms/step - loss: 0.3315 - accuracy: 0.8629
Epoch 132/150
500/500 [==============================] - 1s 2ms/step - loss: 0.3311 - accuracy: 0.8620
Epoch 133/150
500/500 [==============================] - 1s 2ms/step - loss: 0.3306 - accuracy: 0.8624
Epoch 134/150
500/500 [==============================] - 1s 1ms/step - loss: 0.3310 - a

RandomizedSearchCV(cv=5,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000002049293A880>,
                   param_distributions={'batch_size': [16, 32, 64],
                                        'epochs': [50, 100, 150]},
                   scoring='accuracy')

In [29]:
# Access the best parameters
best_params = search1.best_params_
print("Best Parameters:", best_params)

Best Parameters: {'epochs': 150, 'batch_size': 16}


In [30]:
# Access the best score
best_score = search.best_score_
print("Best score:", best_score)

Best score: 0.86075
